# Direct Preference Optimization (DPO) Fine-Tuning Template





Supervised fine-tuning (SFT) is the first step of (DPO).During the fine-tuning phase, DPO uses the LLM as a reward model. It optimizes the policy using a binary cross-entropy objective, leveraging human preference data to determine which responses are preferred and which are not. By comparing the model's responses to the preferred ones, the policy is adjusted to enhance its performance.

`I prepared this Direct Preference Optimization (DPO) template for my use case, but you could change it to suit your requirements.`



To View My Account:

* [Hugging Face ](https://huggingface.co/santhoshmlops)

* [Git Hub](https://github.com/santhoshmlops)

To View Some other Fine Tuning Template:

* [Fine Tuning Template ](https://github.com/santhoshmlops/MyHF_LLM_FineTuning/tree/main/FineTuningTemplate)


To View My Model Fine Tuning  NoteBook:

* [MY HF LLM Fine-Tuning](https://github.com/santhoshmlops/MyHF_LLM_FineTuning)



## Setting Up on Google Colab
Google Colab provides a convenient, cloud-based environment with access to powerful GPUs like the `T4`. If you choose Colab for this tutorial, make sure to select a GPU runtime by going to `Runtime > Change runtime type > T4 GPU`. This ensures that your notebook has access to the necessary computational resources.

## Setting Up Hugging Face Authentication

On Google Colab, you can safely store your Hugging Face token by using Colab's "Secrets" feature. This can be done by clicking on the "Key" icon in the sidebar, selecting "`Secrets`", and adding a new secret with the name `HF_TOKEN` and your Hugging Face token as the value. This method ensures that your token remains secure and is not exposed in your notebook's code.

# Step 1 - Install the required Python packages

In [ ]:
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U bitsandbytes
!pip install -q -U trl
!pip install -q -U accelerate
!pip install -q -U datasets

# Step 2 - Logging into Hugging Face Hub
Paste the Hugging Face Hub Write API KEY

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# Step 3 - Loading Required Libraries

In [ ]:
import torch
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import get_peft_model, LoraConfig,prepare_model_for_kbit_training
from trl import DPOTrainer
from accelerate import Accelerator

# Step 4 - Setting Model Parameters for DPO
`Note:` The parameter can be changed for fine tuning, or it can be left as it is and filled with the value of the empty parameter.

In [ ]:
dpo_config = {
            # Load Model for Tuning
            "model_ckpt": "",  # Use SFT Trained model to perform DPO Fine Tuning
            "load_in_4bit": True,
            "device_map": {"": Accelerator().local_process_index},
            "torch_dtype": torch.float16,
            "trust_remote_code": True,
            # QLora Parameters
            "use_lora": True,
            "r": 8,
            "lora_alpha": 8,
            "lora_dropout": 0.05,
            "bias": "none",
            "task_type": "CAUSAL_LM",
            "target_modules": ["q_proj","k_proj"],
            # Training Parameters
            "output_dir": "Fine_tuned_model-DPO",
            "per_device_train_batch_size": 5,
            "gradient_accumulation_steps": 1,
            "optim": "paged_adamw_32bit",
            "learning_rate": 2e-4,
            "lr_scheduler_type": "cosine",
            "save_strategy": "epoch",
            "logging_steps": 50,
            "num_train_epochs": 1,
            "max_steps": 500,
            "fp16": True,
            "push_to_hub": True,
            "train_cln_name": "text",
            "packing": False,
            "neftune_noise_alpha": 5,
            # SFT Training Parameter
            "beta": 0.1,
            "loss_type": "kto_pair",
            "max_length": 768,
            "max_prompt_length": 512,
            "max_target_length": 256,
            "is_encoder_decoder": False
        }

# Step 5 - Loading and Formatting the Dataset
`Note:` Prepare your dataset for fine tuning by defining and formatting it for your use case. The `def create_data():` function is an example for tuning the dataset.

In [ ]:
dataset_name = "Anthropic/hh-rlhf"
def create_data():
  data = load_dataset(dataset_name, split="train")
  data_df = data.to_pandas()
  data_df["prompt"] = data_df["chosen"].apply(lambda x: x.split("Assistant: ")[0])
  data_df["chosen"] = data_df["chosen"].apply(lambda x: "Assistant: "+ x.split("Assistant: ")[-1])
  data_df["rejected"] = data_df["rejected"].apply(lambda x: "Assistant: " + x.split("Assistant: ")[-1])
  data = Dataset.from_pandas(data_df)
  return data

data = create_data()
print(data[0])

# Step 6 - Fine-Tuning with qLora and Supervised Finetuning

In [ ]:
class TrainDPO:

    def __init__(self, data, config):
        """
        Initialize the TrainSFT class with data and configuration parameters.
        """
        self.data = data
        self.config = config

    def prepare_lora_model(self):
        """
        Prepare the model with LoRA (Low-Rank Adaptation) configuration.
        """

        self.lora_config = LoraConfig(
                                    r=self.config["r"],
                                    lora_alpha=self.config["lora_alpha"],
                                    lora_dropout=self.config["lora_dropout"],
                                    bias=self.config["bias"],
                                    task_type=self.config["task_type"],
                                    target_modules=self.config["target_modules"]
                                )
        self.model = get_peft_model(self.model, self.lora_config)
        self.model_ref = get_peft_model(self.model_ref, self.lora_config)

    def load_model_tokenizer(self):
        """
        Load the model and tokenizer with specified configurations.
        """

        self.model = AutoModelForCausalLM.from_pretrained(
                            self.config["model_ckpt"],
                            load_in_4bit=self.config["load_in_4bit"],
                            device_map=self.config["device_map"],
                            torch_dtype=self.config["torch_dtype"]
                        )

        self.model_ref = AutoModelForCausalLM.from_pretrained(
                            self.config["model_ckpt"],
                            load_in_4bit=self.config["load_in_4bit"],
                            device_map=self.config["device_map"],
                            torch_dtype=self.config["torch_dtype"]
                        )
        self.model.config.use_cache=False
        self.model.config.pretraining_tp=1
        self.model = prepare_model_for_kbit_training(self.model)
        if self.config["use_lora"]:
            self.prepare_lora_model()

        self.tokenizer = AutoTokenizer.from_pretrained(self.config["model_ckpt"])
        self.tokenizer.pad_token = self.tokenizer.eos_token

    def set_training_args(self):
        """
        Set up training arguments.
        """

        return TrainingArguments(
                                    output_dir=self.config["output_dir"],
                                    per_device_train_batch_size=self.config["per_device_train_batch_size"],
                                    gradient_accumulation_steps=self.config["gradient_accumulation_steps"],
                                    optim=self.config["optim"],
                                    learning_rate=self.config["learning_rate"],
                                    lr_scheduler_type=self.config["lr_scheduler_type"],
                                    save_strategy=self.config["save_strategy"],
                                    logging_steps=self.config["logging_steps"],
                                    num_train_epochs=self.config["num_train_epochs"],
                                    max_steps=self.config["max_steps"],
                                    fp16=self.config["fp16"],
                                    push_to_hub=self.config["push_to_hub"],
                                    neftune_noise_alpha=self.config["neftune_noise_alpha"]
                                )

    def create_trainer(self):
        """
        Create a trainer for training the model.
        """

        self.load_model_tokenizer()
        if self.config["use_lora"]:
            print(self.model.print_trainable_parameters())
            self.trainer = DPOTrainer(
                                        self.model,
                                        self.model_ref,
                                        args=self.set_training_args(),
                                        train_dataset=self.data,
                                        tokenizer=self.tokenizer,
                                        beta=self.config["beta"],
                                        loss_type=self.config["loss_type"],
                                        max_length=self.config["max_length"],
                                        max_prompt_length=self.config["max_prompt_length"],
                                        max_target_length=self.config["max_target_length"],
                                        is_encoder_decoder=self.config["is_encoder_decoder"]
                                    )
        else:
            self.trainer = DPOTrainer(
                                        self.model,
                                        self.model_ref,
                                        peft_config=self.lora_config,
                                        args=self.set_training_args(),
                                        train_dataset=self.data,
                                        tokenizer=self.tokenizer,
                                        beta=self.config["beta"],
                                        loss_type=self.config["loss_type"],
                                        max_length=self.config["max_length"],
                                        max_prompt_length=self.config["max_prompt_length"],
                                        max_target_length=self.config["max_target_length"],
                                        is_encoder_decoder=self.config["is_encoder_decoder"]
                                    )

    def train_and_save_model(self):
        """
        Train the model and save it.
        """
        self.create_trainer()
        self.trainer.train()
        self.trainer.save_model(self.config["output_dir"])
        self.tokenizer.save_pretrained(self.config["output_dir"])


# Step 7 - Lets start the training process

In [ ]:
train_dpo = TrainDPO(data, dpo_config)
train_dpo.train_and_save_model()

# Step 8 - Merge the model with LoRA weights

In [ ]:
def merge_push_to_hub(config):

    # Initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained(config["model_ckpt"])
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    # Load base model
    base_model = AutoModelForCausalLM.from_pretrained(
        config["model_ckpt"],
        low_cpu_mem_usage=config["low_cpu_mem_usage"],
        return_dict=config["return_dict"],
        torch_dtype=config["torch_dtype"],
        device_map=config["device_map"]
    )

    # Merge models
    merged_model = PeftModel.from_pretrained(base_model,config["hub_model_ckpt"], from_transformers=True)
    merged_model = merged_model.merge_and_unload()

    # Push the model and tokenizer to the Hugging Face Model Hub
    merged_model.push_to_hub(config["new_model_ckpt"], use_temp_dir=False)
    tokenizer.push_to_hub(config["new_model_ckpt"], use_temp_dir=False)

# Assuming sft_config is defined elsewhere
merge_push_to_hub(sft_config)